In [ ]:
import matplotlib
import pandas as pd
import numpy as np
import json
import math
import urllib.request
import dateutil.parser
import dateutil.rrule
import dateutil.tz
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from textwrap import wrap

matplotlib.rcParams.update({
    'font.size': 13,
    'timezone': 'Europe/London'
})

In [ ]:
# Used across all of the plots
tzLocal = dateutil.tz.gettz('Europe/London')
dateToday = datetime.datetime.combine(datetime.date.today(), datetime.datetime.min.time()).replace(tzinfo=tzLocal)

colourUp = '#f64a8a'
colourDown = '#233067'

resampleFrequency = 900

# Car park occupancy across Tyne and Wear

The data represents the car parks with ANPR at the entry and exit, with data collected through Tyne and Wear UTMC. This is not all car parks.


In [ ]:
dfCarParks = pd.read_pickle('../cache/recent-car-park-occupancy-pd.pkl')
carParkMetadata = pd.read_pickle('../cache/recent-car-park-metadata-pd.pkl')
print('Last data obtained %s' 
    % (np.max(dfCarParks.index).strftime('%d %B %Y %H:%M')))

In [ ]:
def plotCarParkTimeseries(type='continuous', historicCutOffDays=28):
    dfCarParksRecent = dfCarParks[dfCarParks.index >= dateToday - pd.Timedelta(days=historicCutOffDays)]
    dfCarParkMissingFreq = dfCarParksRecent \
          .isna() \
          .sum(axis=0) \
          .apply(lambda c: c / len(dfCarParksRecent.index))
    dfCarParkPlotList = sorted(
          dfCarParkMissingFreq[dfCarParkMissingFreq < 0.5].index,
          key=lambda carPark: carParkMetadata[carParkMetadata.index == carPark]['capacity'].values[0],
          reverse=True
    )

    if type == 'daily':
         dfCarParksRecent = dfCarParksRecent.resample('24H').sum() / (3600 / resampleFrequency)

    fig, axs = plt.subplots(
         len(dfCarParkPlotList), 1,
         figsize = (18, 1.5 * len(dfCarParkPlotList))
    )

    plotIndex = 0

    for carPark in dfCarParkPlotList:
        carParkMeta = carParkMetadata[carParkMetadata.index == carPark]
        ax = axs[plotIndex]
        ax.set_title(
             '%s (%s with %u spaces)' % (
                  carPark,
                  carParkMeta['district'].values[0],
                  carParkMeta['capacity'].values[0]
             ),
             loc='left',
             fontdict={ 'horizontalalignment': 'left', 'fontsize': 12 }
        )
        ax.margins(x=0, y=0)
        if type == 'daily':
            dfWeekdays = dfCarParksRecent[dfCarParksRecent.index.to_series().apply(
                  lambda t: t.strftime('%A') not in ['Saturday', 'Sunday']
            )][carPark]
            dfWeekends = dfCarParksRecent[dfCarParksRecent.index.to_series().apply(
                  lambda t: t.strftime('%A') in ['Saturday', 'Sunday']
            )][carPark]

            ax.bar(
                 dfWeekdays.index,
                 dfWeekdays,
                 color=colourDown,
                 label='Vehicle-hours on weekdays'
            )
            ax.bar(
                 dfWeekends.index,
                 dfWeekends,
                 color=colourUp,
                 label='Vehicle-hours on weekends'
            )
        elif type == 'continuous':
            ax.fill_between(
                dfCarParksRecent[carPark].index,
                dfCarParksRecent[carPark],
                color=colourDown,
                where=dfCarParksRecent.index.to_series().apply(
                     lambda t: t.strftime('%A') not in ['Saturday', 'Sunday']
                ),
                label='Occupied spaces on weekdays'
            )
            ax.fill_between(
                dfCarParksRecent[carPark].index,
                dfCarParksRecent[carPark],
                color=colourUp,
                where=dfCarParksRecent.index.to_series().apply(
                     lambda t: t.strftime('%A') in ['Saturday', 'Sunday']
                ),
                label='Occupied spaces on weekend'
            )

        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1, byweekday=mdates.MO))
        ax.xaxis.set_tick_params(which='major', pad=15)
        ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))

        if ax == axs[-1]:
            ax.set_xlabel('Date')

            if historicCutOffDays > 75:
                timeLocatorMajor = mdates.AutoDateLocator(minticks=10, maxticks=30)
                conciseZeroFormats = ['', '%Y', '%b', '%d-%b', '%H:%M', '%H:%M']
                conciseOffsetFormats = ['', '%Y', '%b-%Y', '%d-%b-%Y-%b', '%d-%b-%Y', '%d-%b-%Y %H:%M']
                ax.xaxis.set_tick_params(which='major', pad=0)
                ax.xaxis.set_major_locator(timeLocatorMajor)
                ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator=timeLocatorMajor, zero_formats=conciseZeroFormats, offset_formats=conciseOffsetFormats))
            else:
                dataFormatMajor = mdates.DateFormatter('%a %d %b')
                ax.xaxis.set_major_formatter(dataFormatMajor)
                ax.xaxis.set_minor_formatter(mdates.DateFormatter('%d'))
        else:
            ax.xaxis.set_ticklabels([]);

        if ax == axs[0]:
            ax.legend(
                loc='upper right',
                ncol=2,
                fontsize=11,
                frameon=False,
                bbox_to_anchor=(1.0, 1.35)
            )

        plotIndex = plotIndex + 1

    plt.tight_layout()
    fig.subplots_adjust(hspace=0.4)
    plt.show()

## Daily total vehicle-hours during the last six months

The below charts are expressed in vehicle-hours, meaning each vehicle being parked is multiplied by the duration of its stay.

In [ ]:
# Daily car-hours as a bar chart
plotCarParkTimeseries('daily', 185)